In [168]:
"""
Скрипт для составления списка общежитий
"""

'\nСкрипт для составления списка общежитий\n'

In [169]:
import pandas as pd
import openpyxl
import time
from openpyxl.utils.dataframe import dataframe_to_rows

In [170]:
ab_df = pd.read_excel('data/Абитуриенты.xlsx',sheet_name='Абитуриенты',skiprows=8,dtype={'ИНН':str,'Серия УЛ':str,'Номер УЛ':str})
#pers_df = pd.read_excel('data/Абитуриенты.xlsx',sheet_name='Персоны',skiprows=2,dtype={'ИНН':str,'Серия УЛ':str,'Номер УЛ':str})

In [171]:
ab_df.head()

,№,ФИО,Пол,Дата рождения,Тип УЛ,Серия УЛ,Номер УЛ,Дата выдачи УЛ,Кем выдано УЛ,ИНН,...,Статус результатов ЕГЭ,Баллы за индивидуальные достижения,Сумма баллов за индивидуальные достижения,"Баллы за индивидуальные достижения, учитываемые как преимущество","Сумма баллов за индивидуальные достижения, учитываемые как преимущество",Организация-заказчик,Целевая контрактная подготовка,Uid ЕПГУ,Договор заключен,Договор оплачен
0,1,Абросова Анастасия Витальевна,Ж,08.07.2007,Паспорт РФ,8112,927500,04.08.2021,МВД ПО РЕСПУБЛИКЕ БУРЯТИЯ,030620313718,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
1,2,Абсаматов Вадим Чимид-Цыренович,М,14.12.2007,Паспорт РФ,8120,900692,23.12.2021,МВД ПО РЕСП. БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
2,3,Авдеев Сергей Александрович,М,09.10.2006,Паспорт РФ,8120,896798,23.12.2020,МВД по РЕСПУБЛИКЕ БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
3,4,Авдеев Сергей Александрович,М,09.10.2006,Паспорт РФ,8120,896798,23.12.2020,МВД по РЕСПУБЛИКЕ БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет
4,5,Аверина Вероника Евгеньевна,Ж,03.12.2007,Паспорт РФ,8121,951540,17.12.2021,МВД ПО РЕСП. БУРЯТИЯ,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,нет,NaN,нет,нет


In [172]:
ab_df = ab_df[ab_df['Состояние выбран. конкурса'] == 'Зачислен'] # оставялем только зачисленных


In [173]:
replace_dct = {'Отделение железнодорожного транспорта':'ЖД','Отделение металлообработки':'МО',
               'Отделение энергоснабжения':'ЭО','Хоринский филиал':'ХФ',}

In [174]:
ab_df['Формирующее подр.'] = ab_df['Формирующее подр.'].replace(replace_dct)

In [175]:
unit_lst = ab_df['Формирующее подр.'].unique()

In [176]:
unit_lst

array(['ЖД', 'МО', 'ЭО', 'ХФ'], dtype=object)

In [177]:
dct_unit = {key:None for key in unit_lst}

In [178]:
dct_unit

{'ЖД': None, 'МО': None, 'ЭО': None, 'ХФ': None}

In [179]:
ab_df.shape

(496, 108)

In [180]:
ab_df = ab_df[ab_df['Нуждается в общежитии']=='да']

In [181]:
ab_df.shape

(219, 108)

In [182]:
ab_df = ab_df[['ФИО','Контактный телефон','Мобильный телефон','Адрес регистрации','Формирующее подр.']]

In [183]:
for name in unit_lst:
    temp_df = ab_df[ab_df['Формирующее подр.']== name]
    temp_df.sort_values(by='ФИО',inplace=True)
    dct_unit[name] = temp_df
    
    

C:\Users\1\AppData\Local\Temp\ipykernel_19544\3465281252.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.sort_values(by='ФИО',inplace=True)


In [184]:
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)

In [185]:
wb = openpyxl.Workbook()
# Переименовываем лист
sheet = wb['Sheet']
sheet.title = 'ЖД'
wb.create_sheet('МО',index=1)
wb.create_sheet('ЭО',index=2)
wb.create_sheet('ХФ',index=3)

<Worksheet "ХФ">

In [186]:
wb.sheetnames

['ЖД', 'МО', 'ЭО', 'ХФ']

In [187]:
for name,df in dct_unit.items():    
    for r in dataframe_to_rows(df,index =False,header=True):
        if len(r) != 1:
            wb[name].append(r)

In [188]:
for name in unit_lst:
    wb[name].column_dimensions['A'].width =40
    wb[name].column_dimensions['B'].width =10
    wb[name].column_dimensions['C'].width =20
    wb[name].column_dimensions['D'].width =90


In [189]:
# Сохраняем итоговый файл
wb.save(f'data/Список на заселение от {current_time}.xlsx')

In [190]:
#pers_df = pers_df[['ФИО','Контактный телефон','Мобильный телефон','Адрес регистрации']] # получаем маленький датафрейм